In [1]:
import itertools
import random

import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [2]:
import flor
flor.__file__

'/Users/rogarcia/git/flor/flor/__init__.py'